### Install mrcnn librarary

In [1]:
#install mrcnn library
!pip3 install mrcnn

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


#### Importing Libraries

In [1]:
import sys
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn import model
import mrcnn.model as modellib
from mrcnn.model import log
import cv2
import os

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

#place custom_1.py inside the folder where this ipynb is run
from samples.roads import road 

/home/lois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lois/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

### Ground Truth Data Generation

In [2]:
# Load dataset
dataset = road.RoadDataset()
custom_DIR = os.path.join(ROOT_DIR, "samples/roads/assets/in_use/aug_roads_test_split")
#place your json file inside train folder
dataset.load_road(custom_DIR, "test")
# Must call before using the dataset
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

Image Count: 392
Class Count: 2
  0. BG                                                
  1. road                                              


In [3]:
#image_ids = np.random.choice(dataset.image_ids, 60)
os.chdir("assets/in_use/aug_roads_test_split/test_masked/test_piece_masked")
index = dataset.image_ids
#index=range(1,5)
#print(image_ids)
to_display = []
titles = []
for image_id in index:
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    #mask=mask[:,:,-1]
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    #print(unique_class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]]) for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),key=lambda r: r[1], reverse=True) if v[1] > 0]
    
    # Generate images and titles
    for i in range(1):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] +1), -1)
        m[m>1]=1
        #print(np.unique(m))
        to_display.append(m)
        titles.append(dataset.class_names[class_id] if class_id != -1 else "-")
        mask = dataset.image_info[image_id]['id']
        #print(m.shape)
        plt.imsave(mask,m)

### Read the binary masked image using Opencv
##### Ideally, background will be having the value of 30 and foreground with value of 215.
##### Inorder to make it to 0's and 1's , we will replace 30 with 0 and 215 with 1.

In [5]:
a=cv2.imread('tile_7605.png',0)
print('Befpre Conversion',np.unique(a))
print(a.shape)
a[a==30]=0
a[a==215]=1### Read the binary masked image using Opencv
##### Ideally, background will be having the value of 30 and foreground with value of 215.
##### Inorder to make it to 0's and 1's , we will replace 30 with 0 and 215 with 1.
print('After Conversion',np.unique(a))
#plt.imshow(a)

Befpre Conversion [None]


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
a=cv2.imread("/home/lois/Desktop/Github Repositories/Mask_RCNN/samples/roads/assets/black.png",0)


In [ ]:
print('Befpre Conversion',np.unique(a))
a[a==0]=1
a[a==255]=0
### Read the binary masked image using Opencv
##### Ideally, background will be having the value of 30 and foreground with value of 215.
##### Inorder to make it to 0's and 1's , we will replace 30 with 0 and 215 with 1.
print('After Conversion',np.unique(a))
#plt.imshow(a)